# GeoMX DCC counts QC for project {{ PROJECT_IGF_ID }}
* **Notebook version:** v0.0.1
* **Created by:** Imperial BRC Genomics Facility
* **Maintained by:** Imperial BRC Genomics Facility
* **Docker image path:** [Dockerfile](https://github.com/imperial-genomics-facility/igf-dockerfiles/tree/main/bioconductor-geomxworkflows/Dockerfile_v1)
* **Notebook code path:** [Templates](https://github.com/imperial-genomics-facility/igf-dockerfiles/tree/main/bioconductor-geomxworkflows/templates)
* **Created on:** {{ DATE_TAG }}
* **Contact us:** [Imperial BRC Genomics Facility](https://www.imperial.ac.uk/medicine/research-and-impact/facilities/genomics-facility/contact-us/)
* **License:** Apache [License 2.0](https://github.com/imperial-genomics-facility/scanpy-notebook-image/blob/master/LICENSE)
* **Project name:** {{ PROJECT_IGF_ID }}
* **Analysis name:** {{ ANALYSIS_NAME }}

Send us your suggestions (or PRs) about how to improve this notebook.

Please add the following statement in all publications rif you use any part of this notebook for your analysis: _“The Imperial BRC Genomics Facility has provided resources and support that have contributed to the research results reported within this paper. The Imperial BRC Genomics Facility is supported by NIHR funding to the Imperial Biomedical Research Centre”._

## Required inputs

Following input files are required for running this notebook:
1. **GeoMx DCC count directory:** _{{ GEOMX_DCC_DIR }}_
2. **GeoMx annotation file:** _{{ GEOMX_ANNOTATION_FILE }}_
3. **PKC file (probe assay metadata describing the gene targets present in the data):** _{{ GEOMX_PKC_FILE }}_


## Table of contents
* [Line chart of read count data](#Line-chart-of-read-count-data)
* [Read count table](#Read-count-table)
* [Loading Data to GeoMx workflows](#Loading-Data-to-GeoMx-workflows)
* [QC & Pre-processing](#QC-&-Pre-processing)
* [Segment QC](#Segment-QC)
* [Visualize Segment QC](#Visualize-Segment-QC)
* [Display samples with warnings](#Display-samples-with-warnings)
* [Remove flagged segments](#Remove-flagged-segments)
* [Probe QC](#Probe-QC)
* [Create Gene-level Count Data](#Create-Gene-level-Count-Data)
* [Limit of Quantification](#Limit-of-Quantification)
* [Normalization](#Normalization)
* [Unsupervised Analysis](#Unsupervised-Analysis)
* [Convert to Seurat object](#Convert-to-Seurat-object)

In [ ]:
## loading Python library
import os
import numpy as np
import altair as alt
import pandas as pd
import scanpy as sc
from IPython.display import HTML
from bs4 import BeautifulSoup
from io import StringIO
alt.renderers.enable("html")

In [ ]:
## anotation processing
def reformat_roi_id_and_scan_name(s):
    if s['Scan Name'] is None or \
       s['Scan Name'] == 'nan':
        s['Scan Name'] = ''
    if s['Roi'] is None or \
       s['Roi'] == 'nan':
        s['Roi'] = ''
    new_roi_id = ''
    roi_id = s['Roi']
    scan_name = s['Scan Name']
    if roi_id != '' and scan_name != '':
        roi_id = roi_id.replace('\"', '').replace('=', '')
        new_roi_id = scan_name + '-' + roi_id
    s['Roi'] = new_roi_id
    return s

def get_formatted_xlsx_for_geomx_workflow(labworksheet_file: str, xlsx_file: str) -> None:
    try:
        annotation_data = list()
        annotation_found = False
        with open(labworksheet_file, 'r') as fp:
            for line in fp:
                if line.startswith('Annotations'):
                    annotation_found = True
                if annotation_found and \
                   not line.startswith('Annotations'):
                    annotation_data.append(line.strip())

        annotation_data = '\n'.join(annotation_data)
        csv_data = StringIO(annotation_data)
        df = pd.read_csv(csv_data, sep='\t', header=0)
        df['Roi'] = df['Roi'].astype(str)
        df['Scan Name'] = df['Scan Name'].astype(str)
        #df['Roi'].fillna('', inplace=True)
        #df['Scan Name'].fillna('', inplace=True)
        df = df.apply(lambda s: reformat_roi_id_and_scan_name(s), axis=1)
        df.to_excel(xlsx_file, sheet_name="Annotation template", index=False)
    except Exception as e:
        raise ValueError(e)

In [ ]:
get_formatted_xlsx_for_geomx_workflow(
    labworksheet_file='{{ GEOMX_ANNOTATION_FILE }}',
    xlsx_file='/tmp/reformatted_annotation.xlsx')

In [ ]:
## get dcc data and table

In [ ]:
def combine_info_from_dcc_and_stats_file(dcc_file: str, stats_file: str) -> dict:
    try:
        dcc_data = dict()
        ## read dcc filet
        with open(dcc_file, 'r') as fp:
            dcc_file_data = fp.read()
        soup = BeautifulSoup(dcc_file_data)
        ## get ngs_attributes
        ngs_attributes = \
            soup.find_all('ngs_processing_attributes')
        if len(ngs_attributes) == 0:
            raise ValueError('No ngs_processing_attributes found')
        ngs_attributes = \
            ngs_attributes[0].contents
        if isinstance(ngs_attributes, list):
            ngs_attributes = \
                ngs_attributes[0]
        ngs_attributes = [
            c.split(',')
                for c in ngs_attributes.split('\n')
                    if c != '']
        for entry in ngs_attributes:
            dcc_data.update({entry[0]: entry[1]})
        ## get scan_attributes
        scan_attributes =\
            soup.find_all('scan_attributes')
        if len(scan_attributes) == 0:
            raise ValueError('No scan_attributes found')
        scan_attributes = \
            scan_attributes[0].contents
        if isinstance(scan_attributes, list):
            scan_attributes = \
                scan_attributes[0]
        scan_attributes = [
            c.split(',')
                for c in scan_attributes.split('\n')
                    if c != '']
        for entry in scan_attributes:
            dcc_data.update({entry[0]: entry[1]})
        ## read stats file
        if os.path.exists(stats_file):
            dedup_entry = ''
            with open(stats_file, 'r') as fp:
                for line in fp:
                    if line.startswith('Reads after dedup'):
                        dedup_entry = line.strip()
                        break
            if len(dedup_entry.split(':')) < 2:
                raise ValueError('No dedup entry found: {dedup_entry}')
            dedup_entry = \
                dedup_entry.split(':')[1]
            dedup_entry = \
                dedup_entry.replace(',', '')
            dcc_data.update({'Dedup': dedup_entry})
        return dcc_data
    except Exception as e:
        raise ValueError(e)

In [ ]:
def style_filter_read(s: pd.Series, props: str, cut_off: int) -> pd.Series:
    return np.where(s <= cut_off, props, '')

def calculate_saturation(s):
    saturation = 0
    if s['Aligned'] > 0:
        saturation = (1 - s['Dedup'] / s['Aligned']) * 100
    s['saturation'] = saturation
    return s

def calculate_alignment_pct(s):
    pct = 0
    if s['Aligned'] > 0:
        pct = s['Aligned'] / s['Raw'] * 100
    s['aligned_pct'] = pct
    return s
    
def generate_read_count_plot_and_table(dcc_base_path: str) -> None:
    all_dcc_data = list()
    for entry in os.listdir(dcc_base_path):
        if entry.endswith('.dcc'):
            dcc_path = os.path.join(dcc_base_path, entry)
            stats_file = dcc_path.replace('.dcc', '.stats')
            entry_dcc_data = \
                combine_info_from_dcc_and_stats_file(
                    dcc_file=dcc_path,
                    stats_file=stats_file)
            all_dcc_data.append(entry_dcc_data)
    df = pd.DataFrame(all_dcc_data)
    df = df[['ID', 'Raw', 'Trimmed', 'Stitched', 'Aligned', 'Dedup']]
    df2 = df.melt('ID', var_name='count_type', value_name='counts')
    options = df2['count_type'].drop_duplicates().values.tolist()
    selection = alt.selection_multi(fields=['count_type'])
    color = alt.condition(selection, alt.Color('count_type:N'), alt.value('lightgray'))
    make_selector = alt.Chart(df2).mark_rect().encode(x=alt.X('count_type').title('Track type'), color=color).add_params(selection)

    line = \
        alt.Chart(df2).mark_line().encode(
            x=alt.X('ID:N').axis(labels=False).title('Roi'),
            y=alt.Y('counts:Q').title('Number of reads'),
            color=alt.Color('count_type:N').scale(domain=options).title('Track type')).\
        add_params(
            selection
        ).transform_filter(
            selection
        )

    point = \
        alt.Chart(df2).mark_point().encode(
            x=alt.X('ID:N').axis(labels=False).title('Roi'),
            y=alt.Y('counts:Q').title('Number of reads'),
            color=alt.Color('count_type:N').scale(domain=options).title('Track type'),
            tooltip=['ID:N', 'counts:Q', 'count_type:N']).\
        add_params(
            selection
        ).transform_filter(
            selection
        )

    lower = \
        alt.layer(
            line, point).\
        properties(
            width=960, height=400)

    
    plot = alt.vconcat(make_selector, lower)
    df.fillna(0, inplace=True)
    df['Dedup'] = df['Dedup'].astype(int)
    df['Aligned'] = df['Aligned'].astype(int)
    df =  df.apply(lambda s: calculate_saturation(s), axis=1)
    df['Raw'] = df['Raw'].astype(int)
    df =  df.apply(lambda s: calculate_alignment_pct(s), axis=1)
    html = df.set_index('ID').\
        style.\
        apply(style_filter_read, props='color:red;', cut_off=1000, axis=0, subset=['Raw',]).\
         apply(style_filter_read, props='background-color:#ffffb3;', cut_off=1000, axis=0, subset=['Raw',]).\
         apply(style_filter_read, props='color:red;', cut_off=80.0, axis=0, subset=['aligned_pct',]).\
         apply(style_filter_read, props='background-color:#ffffb3;', cut_off=80.0, axis=0, subset=['aligned_pct',]).\
         apply(style_filter_read, props='color:red;', cut_off=50.0, axis=0, subset=['saturation',]).\
         apply(style_filter_read, props='background-color:#ffffb3;', cut_off=50.0, axis=0, subset=['saturation',]).\
         to_html()
    return plot, html

In [ ]:
plot, html = generate_read_count_plot_and_table('{{ GEOMX_DCC_DIR }}')

### Line chart of read count data

In [ ]:
plot

### Read count table

In [ ]:
html = '<details><summary>Click to expand summary table</summary>' + html + '</details>'
HTML(html)

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R
suppressMessages(library(NanoStringNCTools))
suppressMessages(library(GeomxTools))
suppressMessages(library(GeoMxWorkflows))
suppressMessages(library(knitr))
suppressMessages(library(dplyr))
suppressMessages(library(ggforce))
suppressMessages(library(ggplot2))
suppressMessages(library(Seurat))
suppressMessages(library(SeuratData))
suppressMessages(library(SeuratDisk))
suppressMessages(library(reshape2))
suppressMessages(library(cowplot))
suppressMessages(library(umap))
suppressMessages(library(Rtsne))
suppressMessages(library(pheatmap))

### Loading Data to GeoMx workflows

* DCCs files - expression count data and sequencing quality metadata
* PKCs file(s) - probe assay metadata describing the gene targets present in the data, PKC files can be found here
* Annotation file - segment area/nuclei count, and other tissue characteristics. If working with a new dataset, use the lab worksheet from the GeoMx instrument study readout package, as the annotation order of NTCs is important to ensure proper processing of files.

All of the expression, annotation, and probe information are now linked and stored together into a single data object.


In [ ]:
%%R
DCCFiles <- dir(file.path("{{ GEOMX_DCC_DIR }}"), pattern = ".dcc$",
                full.names = TRUE, recursive = TRUE)
PKCFiles <- file.path('{{ GEOMX_PKC_FILE }}')
SampleAnnotationFile <-
    file.path('/tmp/reformatted_annotation.xlsx')
# load data
data <-
    readNanoStringGeoMxSet(dccFiles = DCCFiles,
                           pkcFiles = PKCFiles,
                           phenoDataFile = SampleAnnotationFile,
                           phenoDataSheet = "Annotation template",
                           phenoDataDccColName = "Sample_ID",
                           protocolDataColNames = c("Roi", "Aoi"),
                           experimentDataColNames = c("Panel"))

First let’s access the PKC files, to ensure that the expected PKCs have been loaded for this study. For the demo data we are using the input pkc file

In [ ]:
%%R
pkcs <- annotation(data)
modules <- gsub(".pkc", "", pkcs)
kable(data.frame(PKCs = pkcs, modules = modules))

### QC & Pre-processing 
The steps above encompass the standard pre-processing workflow for GeoMx data. In short, they represent the selection of ROI/AOI segments and genes based on quality control (QC) or limit of quantification (LOQ) metrics and data normalization. Before we begin, we will shift any expression counts with a value of 0 to 1 to enable in downstream transformations.


In [ ]:
%%R
# Shift counts to one
data <- shiftCountsOne(data, useDALogic = TRUE)

### Segment QC
Every ROI/AOI segment will be tested for:
* Raw sequencing reads: segments with <1000 raw reads
* % Aligned,% Trimmed, or % Stitched sequencing reads: segments below ~80%
* % Sequencing saturation ([1-deduplicated reads/aligned reads]%): segments below ~50% require additional sequencing to capture full sample diversity and are not typically analyzed until improved.
* Negative Count: this is the geometric mean of the several unique negative probes in the GeoMx panel that do not target mRNA and establish the background count level per segment; segments with low negative counts (1-10) are not necessarily removed but may be studied closer for low endogenous gene signal and/or insufficient tissue sampling
* No Template Control (NTC) count: values >1,000 could indicate contamination for the segments associated with this NTC; however, in cases where the NTC count is between 1,000- 10,000, the segments may be used if the NTC data is uniformly low (e.g. 0-2 counts for all probes).
* Nuclei: >100 nuclei per segment is generally recommended; however, this cutoff is highly study/tissue dependent and may need to be reduced; what is most important is consistency in the nuclei distribution for segments within the study.
* Area: generally correlates with nuclei; a strict cutoff is not generally applied based on area.

In [ ]:
%%R
qc_params <- list(
    minSegmentReads = 1000,
    percentTrimmed = 80,
    percentStiched = 80,
    percentAligned = 80,
    percentSaturation = 50,
    minNegativeCount = 1,
    maxNegativeCount = 1000,
    minNuclei = 100,
    minArea = 1000)

data <- setSegmentQCFlags(
    data,
    qcCutoffs = qc_params)

# Collate QC Results
QCResults <- protocolData(data)[["QCFlags"]]
flag_columns <- colnames(QCResults)
QC_Summary <- data.frame(Pass = colSums(!QCResults[, flag_columns]),
                         Warning = colSums(QCResults[, flag_columns]))
QCResults$QCStatus <- apply(QCResults, 1L, function(x) {
    ifelse(sum(x) == 0L, "PASS", "WARNING")
})
QC_Summary["TOTAL FLAGS", ] <-
    c(sum(QCResults[, "QCStatus"] == "PASS"),
      sum(QCResults[, "QCStatus"] == "WARNING"))

### Visualize Segment QC
Before excluding any low-performing ROI/AOI segments, we visualize the distributions of the data for the different QC parameters. Note that the “Select Segment QC” and “Visualize Segment QC” sections are performed in parallel to fully understand low-performing segments for a given study. Iteration may follow to select the study-specific QC cutoffs.

For QC visualization, we write a quick function to draw histograms of our data.

In [ ]:
%%R
col_by <- "Segment"

# Graphical summaries of QC statistics plot function
QC_histogram <- function(assay_data = NULL,
                         annotation = NULL,
                         fill_by = NULL,
                         thr = NULL,
                         scale_trans = NULL) {
    plt <- ggplot(assay_data,
                  aes_string(x = paste0("unlist(`", annotation, "`)"),
                             fill = fill_by)) +
        geom_histogram(bins = 50) +
        geom_vline(xintercept = thr, lty = "dashed", color = "black") +
        theme_bw() + guides(fill = "none") +
        facet_wrap(as.formula(paste("~", fill_by)), nrow = 4) +
        labs(x = annotation, y = "Segments, #", title = annotation)
    if(!is.null(scale_trans)) {
        plt <- plt +
            scale_x_continuous(trans = scale_trans)
    }
    plt
}

In [ ]:
%%R -w 1480 -h 780 -u px
plot(QC_histogram(sData(data), "Trimmed (%)", col_by, 80))
plot(QC_histogram(sData(data), "Stitched (%)", col_by, 80))
plot(QC_histogram(sData(data), "Aligned (%)", col_by, 80))
plot(QC_histogram(sData(data), "Saturated (%)", col_by, 50) +
   labs(title = "Sequencing Saturation (%)",
        x = "Sequencing Saturation (%)"))
plot(QC_histogram(sData(data), "Area", col_by, 1000, scale_trans = "log10"))
plot(QC_histogram(sData(data), "Nuclei", col_by, 100))

In [ ]:
%%R -w 1480 -h 780 -u px
# calculate the negative geometric means for each module
negativeGeoMeans <- 
    esBy(negativeControlSubset(data), 
         GROUP = "Module", 
         FUN = function(x) { 
             assayDataApply(x, MARGIN = 2, FUN = ngeoMean, elt = "exprs") 
         }) 
protocolData(data)[["NegGeoMean"]] <- negativeGeoMeans

# explicitly copy the Negative geoMeans from sData to pData
negCols <- paste0("NegGeoMean_", modules)
pData(data)[, negCols] <- sData(data)[["NegGeoMean"]]
for(ann in negCols) {
    plt <- QC_histogram(pData(data), ann, col_by, 2, scale_trans = "log10")
    print(plt)
}

In [ ]:
%%R
# detatch neg_geomean columns ahead of aggregateCounts call
pData(data) <- pData(data)[, !colnames(pData(data)) %in% negCols]

# show all NTC values, Freq = # of Segments with a given NTC count:
kable(table(NTC_Count = sData(data)$NTC))

Finally we plot all of the QC Summary information in a table.

In [ ]:
%%R
kable(QC_Summary, caption = "QC Summary Table for each Segment")

### Display samples with warnings

In [ ]:
%%R -o a
a = pData(data[, QCResults$QCStatus == "WARNING"])

In [ ]:
html = a[['Slide Name', 'Scan Name','Segment']].fillna('').to_html()
html = '<details><summary>Click to expand summary table</summary>' + html + '</details>'
HTML(html)

### Remove flagged segments

As the final step in Segment QC, we remove flagged segments that do not meet our QC cutoffs.

In [ ]:
%%R
data <- data[, QCResults$QCStatus == "PASS"]

# Subsetting our dataset has removed samples which did not pass QC
dim(data)

### Probe QC
Before we summarize our data into gene-level count data, we will remove low-performing probes. In short, this QC is an outlier removal process, whereby probes are either removed entirely from the study (global) or from specific segments (local). The QC applies to gene targets for which there are multiple distinct probes representing the count for a gene per segment. In WTA data, one specific probe exists per target gene; thus, Probe QC does not apply to the endogenous genes in the panel. Rather, it is performed on the negative control probes; there are multiple probes representing our negative controls, which do not target any sequence in the genome. These probes enable calculation of the background per segment and will be important for determining gene detection downstream.

After Probe QC, there will always remain at least one probe representing every gene target. In other words, Probe QC never removes genes from your data

#### Set Probe QC Flags

A probe is removed globally from the dataset if either of the following is true:

* the geometric mean of that probe’s counts from all segments divided by the geometric mean of all probe counts representing the target from all segments is less than 0.1
* the probe is an outlier according to the Grubb’s test in at least 20% of the segments

A probe is removed locally (from a given segment) if the probe is an outlier according to the Grubb’s test in that segment.

We do not typically adjust these QC parameters.

In [ ]:
%%R
# Generally keep the qcCutoffs parameters unchanged. Set removeLocalOutliers to 
# FALSE if you do not want to remove local outliers
data <- setBioProbeQCFlags(data, 
                               qcCutoffs = list(minProbeRatio = 0.1,
                                                percentFailGrubbs = 20), 
                               removeLocalOutliers = TRUE)

ProbeQCResults <- fData(data)[["QCFlags"]]

# Define QC table for Probe QC
qc_df <- data.frame(Passed = sum(rowSums(ProbeQCResults[, -1]) == 0),
                    Global = sum(ProbeQCResults$GlobalGrubbsOutlier),
                    Local = sum(rowSums(ProbeQCResults[, -2:-1]) > 0
                                & !ProbeQCResults$GlobalGrubbsOutlier))

In [ ]:
%%R
kable(qc_df)

In [ ]:
%%R
#Subset object to exclude all that did not pass Ratio & Global testing
ProbeQCPassed <- 
    subset(data, 
           fData(data)[["QCFlags"]][,c("LowProbeRatio")] == FALSE &
               fData(data)[["QCFlags"]][,c("GlobalGrubbsOutlier")] == FALSE)

In [ ]:
%%R
dim(ProbeQCPassed)

In [ ]:
%%R
data <- ProbeQCPassed 

### Create Gene-level Count Data

With our Probe QC steps complete, we will generate a gene-level count matrix. The count for any gene with multiple probes per segment is calculated as the geometric mean of those probes.

In [ ]:
%%R
# Check how many unique targets the object has
length(unique(featureData(data)[["TargetName"]]))

In [ ]:
%%R
# collapse to targets
target_data <- aggregateCounts(data)
dim(target_data)

In [ ]:
%%R
exprs(target_data)[1:5, 1:2]

### Limit of Quantification
In addition to Segment and Probe QC, we also determine the limit of quantification (LOQ) per segment. The LOQ is calculated based on the distribution of negative control probes and is intended to approximate the quantifiable limit of gene expression per segment. Please note that this process is more stable in larger segments. Likewise, the LOQ may not be as accurately reflective of true signal detection rates in segments with low negative probe counts (ex: <2). The formula for calculating the LOQ in the ith segment is:

$$LOQi=geomean(NegProbei)∗geoSD(NegProbei)^n$$

We typically use 2 geometric standard deviations (n=2) above the geometric mean as the LOQ, which is reasonable for most studies. We also recommend that a minimum LOQ of 2 be used if the LOQ calculated in a segment is below this threshold.

In [ ]:
%%R
# Define LOQ SD threshold and minimum value
cutoff <- 2
minLOQ <- 2

# Calculate LOQ per module tested
LOQ <- data.frame(row.names = colnames(target_data))
for(module in modules) {
    vars <- paste0(c("NegGeoMean_", "NegGeoSD_"),
                   module)
    if(all(vars[1:2] %in% colnames(pData(target_data)))) {
        LOQ[, module] <-
            pmax(minLOQ,
                 pData(target_data)[, vars[1]] * 
                     pData(target_data)[, vars[2]] ^ cutoff)
    }
}
pData(target_data)$LOQ <- LOQ

### Filtering LOQ
After determining the limit of quantification (LOQ) per segment, we recommend filtering out either segments and/or genes with abnormally low signal. Filtering is an important step to focus on the true biological data of interest.

We determine the number of genes detected in each segment across the dataset.

In [ ]:
%%R
LOQ_Mat <- c()
for(module in modules) {
    ind <- fData(target_data)$Module == module
    Mat_i <- t(esApply(target_data[ind, ], MARGIN = 1,
                       FUN = function(x) {
                           x > LOQ[, module]
                       }))
    LOQ_Mat <- rbind(LOQ_Mat, Mat_i)
}
# ensure ordering since this is stored outside of the geomxSet
LOQ_Mat <- LOQ_Mat[fData(target_data)$TargetName, ]

In [ ]:
%%R
# Save detection rate information to pheno data
pData(target_data)$GenesDetected <- 
    colSums(LOQ_Mat, na.rm = TRUE)
pData(target_data)$GeneDetectionRate <-
    pData(target_data)$GenesDetected / nrow(target_data)

In [ ]:
%%R
# Determine detection thresholds: 1%, 5%, 10%, 15%, >15%
pData(target_data)$DetectionThreshold <- 
    cut(pData(target_data)$GeneDetectionRate,
        breaks = c(0, 0.01, 0.05, 0.1, 0.15, 1),
        labels = c("<1%", "1-5%", "5-10%", "10-15%", ">15%"))

In [ ]:
%%R -w 800 -h 600 -u px
# stacked bar plot of different cut points (1%, 5%, 10%, 15%)
ggplot(pData(target_data),
       aes(x = DetectionThreshold)) +
    geom_bar(aes(fill = Segment)) +
    geom_text(stat = "count", aes(label = ..count..), vjust = -0.5) +
    theme_bw() +
    scale_y_continuous(expand = expansion(mult = c(0, 0.1))) +
    labs(x = "Gene Detection Rate",
         y = "Segments, #",
         fill = "Segment Type")

Now creating a table of Rois which are going to be impacted by each threshold:

In [ ]:
%%R
kable(table(pData(target_data)$DetectionThreshold,
            pData(target_data)$Segment))

Now remove segments with less than 5% of the genes detected.

In [ ]:
%%R
target_data <-
    target_data[, pData(target_data)$GeneDetectionRate >= .05]

dim(target_data)

### Normalization
We will now normalize the GeoMx data for downstream visualizations and differential expression. The two common methods for normalization of DSP-NGS RNA data are i) quartile 3 (Q3) or ii) background normalization.

Both of these normalization methods estimate a normalization factor per segment to bring the segment data distributions together. More advanced methods for normalization and modeling are under active development. However, for most studies, these methods are sufficient for understanding differences between biological classes of segments and samples.

Q3 normalization is typically the preferred normalization strategy for most DSP-NGS RNA studies. Given the low negative probe counts in this particular dataset as shown during Segment QC, we would further avoid background normalization as it may be less stable.

Before normalization, we will explore the relationship between the upper quartile (Q3) of the counts in each segment with the geometric mean of the negative control probes in the data. Ideally, there should be a separation between these two values to ensure we have stable measure of Q3 signal. If you do not see sufficient separation between these values, you may consider more aggressive filtering of low signal segments/genes.

In [ ]:
%%R -w 1280 -h 800 -u px
# Graph Q3 value vs negGeoMean of Negatives
ann_of_interest <- "Segment"
negativeProbefData <- subset(fData(target_data), CodeClass == "Negative")
neg_probes <- unique(negativeProbefData$TargetName)
Stat_data <- 
    data.frame(row.names = colnames(exprs(target_data)),
               Segment = colnames(exprs(target_data)),
               Annotation = pData(target_data)[, ann_of_interest],
               Q3 = unlist(apply(exprs(target_data), 2,
                                 quantile, 0.75, na.rm = TRUE)),
               NegProbe = exprs(target_data)[neg_probes, ])
Stat_data_m <- melt(Stat_data, measure.vars = c("Q3", "NegProbe"),
                    variable.name = "Statistic", value.name = "Value")
plt1 <- ggplot(Stat_data_m,
               aes(x = Value, fill = Statistic)) +
    geom_histogram(bins = 40) + theme_bw() +
    scale_x_continuous(trans = "log2") +
    facet_wrap(~Annotation, nrow = 1) + 
    scale_fill_brewer(palette = 3, type = "qual") +
    labs(x = "Counts", y = "Segments, #")
plt2 <- ggplot(Stat_data,
               aes(x = NegProbe, y = Q3, color = Annotation)) +
    geom_abline(intercept = 0, slope = 1, lty = "dashed", color = "darkgray") +
    geom_point() + guides(color = "none") + theme_bw() +
    scale_x_continuous(trans = "log2") + 
    scale_y_continuous(trans = "log2") +
    theme(aspect.ratio = 1) +
    labs(x = "Negative Probe GeoMean, Counts", y = "Q3 Value, Counts")
plt3 <- ggplot(Stat_data,
               aes(x = NegProbe, y = Q3 / NegProbe, color = Annotation)) +
    geom_hline(yintercept = 1, lty = "dashed", color = "darkgray") +
    geom_point() + theme_bw() +
    scale_x_continuous(trans = "log2") + 
    scale_y_continuous(trans = "log2") +
    theme(aspect.ratio = 1) +
    labs(x = "Negative Probe GeoMean, Counts", y = "Q3/NegProbe Value, Counts")
btm_row <- plot_grid(plt2, plt3, nrow = 1, labels = c("B", ""),
                     rel_widths = c(0.43,0.57))
plot_grid(plt1, btm_row, ncol = 1, labels = c("A", ""))

Next, we normalize our data. We will use Q3 normalized data moving forward. We use the normalize function from NanoStringNCTools to create normalization factors reflecting each data type. Upper quartile (Q3) normalization is performed using norm_method = "quant" setting the desiredQuantile flag to 0.75. Other quantiles could be specified by changing that value. We save the normalized data to a specific slot using toELT = "q_norm". Similarly background normalization is performed by setting norm_method = "neg" and toElt = "neg_norm".

In [ ]:
%%R
# Q3 norm (75th percentile) for WTA/CTA  with or without custom spike-ins
target_data <- normalize(target_data ,
                          norm_method = "quant", 
                             desiredQuantile = .75,
                             toElt = "q_norm")

# Background normalization for WTA/CTA without custom spike-in
target_data <- normalize(target_data ,
                             norm_method = "neg", 
                             fromElt = "exprs",
                             toElt = "neg_norm")


To demonstrate the effects of normalization, we graph representative box plots of the data for individual segments before and after normalization.

In [ ]:
%%R -w 1080 -h 600 -u px
# visualize the first 10 segments with each normalization method
boxplot(exprs(target_data)[,1:10],
        col = "#9EDAE5", main = "Raw Counts",
        log = "y", names = 1:10, xlab = "Segment",
        ylab = "Counts, Raw")

In [ ]:
%%R -w 1080 -h 600 -u px
boxplot(assayDataElement(target_data[,1:10], elt = "q_norm"),
        col = "#2CA02C", main = "Q3 Norm Counts",
        log = "y", names = 1:10, xlab = "Segment",
        ylab = "Counts, Q3 Normalized")

In [ ]:
%%R -w 1080 -h 600 -u px
boxplot(assayDataElement(target_data[,1:10], elt = "neg_norm"),
        col = "#FF7F0E", main = "Neg Norm Counts",
        log = "y", names = 1:10, xlab = "Segment",
        ylab = "Counts, Neg. Normalized")

### Unsupervised Analysis
#### UMAP & t-SNE
One common approach to understanding high-plex data is dimension reduction. Two common methods are UMAP and tSNE, which are non-orthogonally constrained projections that cluster samples based on overall gene expression. In this study, we see by either UMAP (from the umap package) or tSNE (from the Rtsne package), clusters of segments related to structure (glomeruli or tubules) and disease status (normal or diabetic kidney disease).

In [ ]:
%%R
n_neighbors = 15
s = ncol(target_data)
if (n_neighbors > s){
    n_neighbors <- s
}
n_neighbors

In [ ]:
%%R -w 800 -h 600 -u px
# update defaults for umap to contain a stable random_state (seed)
custom_umap <- umap::umap.defaults
custom_umap$random_state <- 42
custom_umap$n_neighbors <- n_neighbors
# run UMAP
umap_out <-
    umap(t(log2(assayDataElement(target_data , elt = "q_norm"))),  
         config = custom_umap)
pData(target_data)[, c("UMAP1", "UMAP2")] <- umap_out$layout[, c(1,2)]
ggplot(pData(target_data),
       aes(x = UMAP1, y = UMAP2, color = Segment)) +
    geom_point(size = 3) +
    theme_bw()

In [ ]:
%%R -w 800 -h 600 -u px
# run tSNE
set.seed(42) # set the seed for tSNE as well
tsne_out <-
    Rtsne(t(log2(assayDataElement(target_data , elt = "q_norm"))),
          perplexity = ncol(target_data)*.15)
pData(target_data)[, c("tSNE1", "tSNE2")] <- tsne_out$Y[, c(1,2)]
ggplot(pData(target_data),
       aes(x = tSNE1, y = tSNE2, color = Segment)) +
    geom_point(size = 3) +
    theme_bw()

#### Clustering high CV Genes
Another approach to explore the data is to calculate the coefficient of variation (CV) for each gene (g) using the formula: $$CVg=SDg/mean_g$$ We then identify genes with high CVs that should have large differences across the various profiled segments. This unbiased approach can reveal highly variable genes across the study.

We plot the results using unsupervised hierarchical clustering, displayed as a heatmap.

In [ ]:
%%R
# create a log2 transform of the data for analysis
assayDataElement(object = target_data, elt = "log_q") <-
    assayDataApply(target_data, 2, FUN = log, base = 2, elt = "q_norm")
# create CV function
calc_CV <- function(x) {sd(x) / mean(x)}
CV_dat <- assayDataApply(target_data,
                         elt = "log_q", MARGIN = 1, calc_CV)
# show the highest CD genes and their CV values
sort(CV_dat, decreasing = TRUE)[1:5]

In [ ]:
%%R -w 800 -h 600 -u px
# Identify genes in the top 3rd of the CV values
GOI <- names(CV_dat)[CV_dat > quantile(CV_dat, 0.8)]
pheatmap(assayDataElement(target_data[GOI, ], elt = "log_q"),
         scale = "row", 
         show_rownames = FALSE, show_colnames = FALSE,
         border_color = NA,
         clustering_method = "average",
         clustering_distance_rows = "correlation",
         clustering_distance_cols = "correlation",
         color = colorRampPalette(c("purple3", "blue", "yellow2"))(120)
         )

### Convert to Seurat object

In [ ]:
%%R
seuratObj <- as.Seurat(target_data, normData = "q_norm")

In [ ]:
%%R -w 600 -h 500 -u px
#All Seurat functionality is available after coercing. Outputs might differ if the ident value is set or not.
VlnPlot(seuratObj, features = "nCount_GeoMx", pt.size = 0.1)

In [ ]:
%%R
npcs = 50
n_neighbors = 30
s = ncol(target_data)
if (npcs > s){
    npcs <- s - 1
}
if (n_neighbors > s){
    n_neighbors <- s
}

seuratObj <- suppressMessages(FindVariableFeatures(seuratObj, verbose = FALSE))
seuratObj <- suppressMessages(ScaleData(seuratObj))
seuratObj <- suppressMessages(RunPCA(seuratObj, assay = "GeoMx", npcs = npcs, verbose = TRUE))
seuratObj <- suppressMessages(FindNeighbors(seuratObj, verbose = FALSE, reduction = "pca", dims = seq_len(npcs)))
seuratObj <- suppressMessages(FindClusters(seuratObj, verbose = FALSE))
seuratObj <- suppressMessages(RunUMAP(seuratObj, n.neighbors = npcs, reduction = "pca", verbose = FALSE, dims = seq_len(npcs)))

In [ ]:
%%R -w 800 -h 600 -u px
DimPlot(seuratObj, reduction = "umap", label = TRUE, label.size=6)

In [ ]:
%%R -w 800 -h 600 -u px
DimPlot(seuratObj, reduction = "umap", label = TRUE, group.by = "Segment")